In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
print(tf.version)

In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
%cd keras-retinanet/

!pip install .

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
ANNOTATIONS_FILE = '' # Enter your annotations csv file
CLASSES_FILE = '' #Enter your classes csv file
DATASET_DIR = ''
JPEGImages = ''

In [ ]:
import os
import shutil
import zipfile
import urllib
import urllib.request
import xml.etree.ElementTree as ET
import numpy as np
import csv
import pandas
from google.colab import drive
from google.colab import files
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

import tensorflow.compat.v1 as tf
import tensorflow.keras.backend as K

# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

def get_session():
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.compat.v1.Session(config=config)

tf.compat.v1.keras.backend.set_session(get_session)

LOAD TEACHER MODEL

In [ ]:
snap_shots_path = "" # Enter teacher weights snapshots path 
model_path = os.path.join(snap_shots_path, sorted(os.listdir(snap_shots_path), reverse=True)[0])
print(model_path)

# load retinanet model
model1 = models.load_model(model_path, backbone_name='resnet50')
model1 = models.convert_model(model1)

# load label to names mapping for visualization purposes
labels_to_names = pandas.read_csv(CLASSES_FILE,header=None).T.loc[0].to_dict()

MAKE PSUDO LABELS ANNOTATION CSV FILE

In [ ]:
import csv

path1 = DATASET_DIR 
save_path = '' #Enter your save path
image_liste1 = os.listdir(path1)

for j in image_liste1:
  test_img_path1 = path1 + j

  img1 = load_img(test_img_path1)
  image1 = img_to_array(img1)

  # preprocess image for network
  image1 = preprocess_image(image1)
  image1, scale1 = resize_image(image1)

  # process image
  start = time.time()
  boxes1, scores1, labels1 = model1.predict_on_batch(np.expand_dims(image1, axis=0))
  print("processing time: ", time.time() - start)
  # correct for image scale
  boxes1 /= scale1

  for box1, score1, label1 in zip(boxes1[0], scores1[0], labels1[0]):
      if score1 < THRES_SCORE:
          break

      caption2 = "{} {:.3f}".format(labels_to_names[label1], score1)
      with open('annotations_pseudo.csv', mode='a') as csv_file:
        writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([test_img_path1, str(int(box1[0])), str(int(box1[1])), str(int(box1[2])), str(int(box1[3])), caption])

  print(j)